In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from labeling.upload import ETLProcessor, get_spark, get_batch_intervals, sampling, uploading
from omegaconf import OmegaConf

In [3]:
config = OmegaConf.load("config/local_full.yaml")

In [4]:
spark = get_spark()

In [5]:
df = sampling(spark, config, 7059473, 7071323)

Successfully queried data from database


In [6]:
# df.groupBy("block_height").count().count()

In [7]:
dff = ETLProcessor.data_scoring(df, **config.hp.etl)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [8]:
dff.printSchema()

root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- self_bonded: decimal(38,6) (nullable = true)
 |-- delegators_token: decimal(38,6) (nullable = true)
 |-- voting_power_score: decimal(38,6) (nullable = true)
 |-- commission_score: double (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- vote_score: double (nullable = true)
 |-- score: double (nullable = true)
 |-- label: double (nullable = true)



In [9]:
# dff.groupBy("block_height").count().count()

69

In [ ]:
dff.show(50, truncate=False)

In [17]:
uploading(dff, config)

In [16]:
dff.count()

2691